## Examen Bimestral – Diseño de un Sistema Básico de Recuperación de Información

## Nombre: Michael Pillaga
## Grupo: Gr1 Cc

Usamos una base de datos llamada rotten-tomatoes-movies-and-critics-reviews, esta contendra dos archivos .csv, una se encarga de dar las peliculas ("movies") y otro de las criticas ("reviews") para cada una de ellas. Asi el primer paso para este ejercicio es cargar ambos BD.

En esta parte vamos a cargar los datos de ambas BD para su utilizacion en las siguientes partes del ejercicio

In [1]:
# Importar librerías
import pandas as pd
import numpy as np

# Cargar archivos CSV
movies_file = './archive/rotten_tomatoes_movies.csv'  # Ruta actualizada
reviews_file = './archive/rotten_tomatoes_critic_reviews.csv'  # Ruta actualizada

# Leer datos
movies_df = pd.read_csv(movies_file)
reviews_df = pd.read_csv(reviews_file)

# Mostrar tres registros de cada archivo para verificar la carga
print("\nPelículas (tres registros de ejemplo):")
print(movies_df.iloc[:3])  # Mostrar las primeras tres filas del DataFrame de películas

print("\nReseñas de críticos (tres registros de ejemplo):")
print(reviews_df.iloc[:3])  # Mostrar las primeras tres filas del DataFrame de reseñas



Películas (tres registros de ejemplo):
  rotten_tomatoes_link                                        movie_title  \
0            m/0814255  Percy Jackson & the Olympians: The Lightning T...   
1            m/0878835                                        Please Give   
2                 m/10                                                 10   

                                          movie_info  \
0  Always trouble-prone, the life of teenager Per...   
1  Kate (Catherine Keener) and her husband Alex (...   
2  A successful, middle-aged Hollywood songwriter...   

                                   critics_consensus content_rating  \
0  Though it may seem like just another Harry Pot...             PG   
1  Nicole Holofcener's newest might seem slight i...              R   
2  Blake Edwards' bawdy comedy may not score a pe...              R   

                                              genres          directors  \
0  Action & Adventure, Comedy, Drama, Science Fic...     Chris Col

## Preprocesamiento
Este código procesa texto contenido en un dataframe (reviews_df) para limpiar y preparar la información. Usa las librerías pandas para manejar los datos, string para eliminar puntuación, y nltk para dividir texto en palabras (tokenización) y simplificarlas a su raíz (stemming). Además, define una lista de palabras comunes (stopwords) que se eliminan por ser irrelevantes. 

El flujo principal consiste en: 
1. Convertir texto a minúsculas.
2. Eliminar signos de puntuación.
3. Tokenizar el texto en palabras individuales.
4. Quitar palabras irrelevantes de la lista de stopwords.
5. Aplicar stemming para reducir palabras a su forma base.

El resultado procesado se guarda en una nueva columna llamada processed_review_content. Finalmente, se imprime una muestra comparando el texto original con el procesado, siempre que la columna review_content exista en el dataframe.


In [5]:
# Importar librerías necesarias
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

# Crear instancia de SnowballStemmer
stemmer = SnowballStemmer("english")  # Stemming en inglés

# Definir stopwords manualmente
custom_stopwords = set([
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he',
    'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'
])

# Función para eliminar signos de puntuación
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Función para procesar texto
def preprocess_text_with_stemming(text):
    if pd.isnull(text):  # Manejar valores nulos
        return ''
    text = text.lower()  # Convertir a minúsculas
    text = remove_punctuation(text)  # Eliminar signos de puntuación
    tokens = word_tokenize(text)  # Tokenizar texto
    tokens = [word for word in tokens if word not in custom_stopwords]  # Eliminar stopwords
    stemmed_tokens = [stemmer.stem(word) for word in tokens]  # Aplicar stemming
    return ' '.join(stemmed_tokens)

# Procesar columnas relevantes
# Preprocesar la columna 'review_content' de reviews_df
if 'review_content' in reviews_df.columns:
    reviews_df['processed_review_content'] = reviews_df['review_content'].apply(preprocess_text_with_stemming)
    print("\nColumna 'processed_review_content' creada con éxito.")
else:
    print("La columna 'review_content' no existe en reviews_df")

# Mostrar resultados de ejemplo
print("\nReseñas ('review_content' procesada, ejemplo):")
if 'processed_review_content' in reviews_df.columns:
    print(reviews_df[['review_content', 'processed_review_content']].head(3))
else:
    print("No hay datos procesados en reviews_df")



Columna 'processed_review_content' creada con éxito.

Reseñas ('review_content' procesada, ejemplo):
                                      review_content  \
0  A fantasy adventure that fuses Greek mythology...   
1  Uma Thurman as Medusa, the gorgon with a coiff...   
2  With a top-notch cast and dazzling special eff...   

                            processed_review_content  
0  fantasi adventur fuse greek mytholog contempor...  
1  uma thurman medusa gorgon coiffur writh snake ...  
2  topnotch cast dazzl special effect this tide t...  


## Vectorizacion
Este código utiliza la técnica TF-IDF para transformar un conjunto de reseñas procesadas en representaciones numéricas.

1. **Preparación del Corpus**:  
   - Verifica si la columna processed_review_content existe en el dataframe reviews_df.
   - Si existe, convierte las reseñas procesadas en una lista (reviews_corpus) y almacena los identificadores (rotten_tomatoes_link) en otra lista.

2. **Inicialización del Vectorizador TF-IDF**:  
   - Usa TfidfVectorizer de sklearn para analizar la relevancia de términos en el corpus.

3. **Ajuste y Transformación**:  
   - Ajusta el vectorizador al corpus (fit_transform) para generar una matriz de características que representa la relevancia de los términos en las reseñas.

4. **Análisis de Términos**:  
   - Extrae y muestra la cantidad total de términos únicos generados y un ejemplo de los 10 primeros términos destacados del corpus.

5. **Transformación en DataFrame (Opcional)**:  
   - Convierte la matriz TF-IDF en un dataframe para facilitar la inspección y manipulación de las características.

6. **Vinculación con Identificadores**:  
   - Crea un nuevo dataframe que relaciona cada reseña (documento) con su enlace identificador (rotten_tomatoes_link) y su vector TF-IDF.

7. **Resultados**:  
   - Imprime ejemplos del dataframe de características TF-IDF (primeras 5 reseñas) y la vinculación de identificadores con sus vectores correspondientes.

Este flujo permite convertir un corpus textual en datos numéricos listos para análisis o modelos de aprendizaje automático, mientras se mantiene la trazabilidad entre las reseñas y sus identificadores únicos.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Verificar si la columna 'processed_review_content' existe
if 'processed_review_content' in reviews_df.columns:
    # Crear el corpus y guardar identificadores
    reviews_corpus = reviews_df['processed_review_content'].dropna().tolist()
    reviews_links = reviews_df['rotten_tomatoes_link'].dropna().tolist()
else:
    raise ValueError("La columna 'processed_review_content' no existe en reviews_df.")

# Inicializar el vectorizador TF-IDF con parámetros optimizados
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,          # Limitar a los 10,000 términos más importantes
    max_df=0.9,                  # Excluir términos que aparecen en más del 90% de los documentos
    min_df=5,                    # Excluir términos que aparecen en menos de 5 documentos
    token_pattern=r'\b[a-zA-Z]+\b'  # Considerar solo palabras con letras (sin números)
)

# Ajustar el vectorizador al corpus y transformar las reseñas en una matriz TF-IDF
tfidf_matrix_reviews = tfidf_vectorizer.fit_transform(reviews_corpus)

# Vincular cada documento del TF-IDF con su identificador
reviews_identifiers = pd.DataFrame({
    'rotten_tomatoes_link': reviews_links[:1000],  # Usar solo los primeros 1000 enlaces
    'tfidf_vector': list(tfidf_matrix_reviews[:1000])  # Guardar los vectores como columna
})

# Mostrar solo un ejemplo de identificadores y vectores TF-IDF
print("\nEjemplo de identificadores y vectores TF-IDF (primeros 5 documentos):")
print(reviews_identifiers.head())



Ejemplo de identificadores y vectores TF-IDF (primeros 5 documentos):
  rotten_tomatoes_link                                       tfidf_vector
0            m/0814255    (0, 8178)\t0.24835750855518937\n  (0, 9542)\...
1            m/0814255    (0, 772)\t0.39144684145955144\n  (0, 8838)\t...
2            m/0814255    (0, 4456)\t0.2912069282471908\n  (0, 6602)\t...
3            m/0814255    (0, 3339)\t0.23822254211838345\n  (0, 997)\t...
4            m/0814255    (0, 150)\t0.23430970512630783\n  (0, 580)\t0...


## Simular las consultas
Este código es como un buscador que te ayuda a encontrar películas basadas en una palabra o frase que escribas. Lo que hace es comparar tu búsqueda con las reseñas de películas, usando una técnica llamada "tecnica de coseno" para ver qué tan parecidas son. Primero convierte tu búsqueda en números (con un modelo llamado TF-IDF), luego compara esos números con las reseñas, ordena las más parecidas, y las conecta con las películas correspondientes. Al final, te muestra las 10 películas más relacionadas con tu búsqueda, junto con sus reseñas.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def simulate_query_reviews_to_movies(query, reviews_df, movies_df, tfidf_vectorizer, tfidf_matrix_reviews):
  
    # Procesar la consulta
    processed_query = preprocess_text_with_stemming(query)

    # Vectorizar la consulta
    query_vector = tfidf_vectorizer.transform([processed_query])

    # Calcular similitudes coseno con las reseñas
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix_reviews).flatten()

    # Obtener índices ordenados de las reseñas relevantes
    related_docs_indices = cosine_similarities.argsort()[::-1]

    # Crear un DataFrame con resultados relevantes de reseñas
    relevant_reviews = reviews_df.iloc[related_docs_indices].copy()
    relevant_reviews['relevance'] = cosine_similarities[related_docs_indices]

    # Filtrar reseñas con relevancia mayor a 0 y vincular con las películas
    relevant_reviews = relevant_reviews[relevant_reviews['relevance'] > 0]
    merged_results = relevant_reviews.merge(
        movies_df, on='rotten_tomatoes_link', how='inner'
    )

    # Ordenar resultados por relevancia y devolver las 10 mejores
    merged_results = merged_results.sort_values(by='relevance', ascending=False)
    return merged_results[['movie_title', 'review_content', 'processed_review_content', 'relevance']].head(10)

# Ejecutar la consulta
query = "horror"
relevant_movies = simulate_query_reviews_to_movies(
    query=query,
    reviews_df=reviews_df,
    movies_df=movies_df,
    tfidf_vectorizer=tfidf_vectorizer,
    tfidf_matrix_reviews=tfidf_matrix_reviews
)

# Mostrar resultados
print("\nPelículas más relevantes para la consulta:")
print(relevant_movies)



Películas más relevantes para la consulta:
                  movie_title                       review_content  \
0                      Casper              The horror, the horror.   
14                        8MM              The horror. The horror.   
45          Stephen King's It                   Top-Calibre horror   
44          Stephen King's It                   Top-Calibre horror   
1   The Bride of Frankenstein               A horror mega-classic.   
22                 Black Swan           Retarded ballerina horror.   
15          Alone in the Dark             The horror. The hor-ror!   
52                 The Rental             Airbnb: The horror movie   
91              Sleepy Hollow  A horror story without much horror.   
92              Sleepy Hollow  A horror story without much horror.   

            processed_review_content  relevance  
0                      horror horror   1.000000  
14                     horror horror   1.000000  
45                  topcalibr horro

## Analisis de resultados

Aqui en el analisis de resultados se puede observar que se tiene peliculas de horror con tambien el termino de horror